In [30]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")

print("Path to dataset files:", path)

Path to dataset files: /home/pasha/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/8


In [31]:
import os
import cv2
import numpy as np
import pandas as pd
import warnings

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict

# Suppress warnings for better output
warnings.filterwarnings("ignore", category=UserWarning)

classes = os.listdir(os.path.join(path, "garbage-dataset"))
print(classes)

['trash', 'glass', 'battery', 'clothes', 'metal', 'plastic', 'cardboard', 'paper', 'biological', 'shoes']


## Извлечение фичей

In [32]:
def calculate_yellow_amount(image):
    """Calculate the amount of yellow color in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    yellow_amount = cv2.countNonZero(yellow_mask)
    return yellow_amount

def calculate_silver_amount(image):
    """Calculate the amount of silver color in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_silver = np.array([0, 0, 200])
    upper_silver = np.array([180, 25, 255])
    silver_mask = cv2.inRange(hsv_image, lower_silver, upper_silver)
    silver_amount = cv2.countNonZero(silver_mask)
    return silver_amount

def calculate_parallel_lines(image):
    """Calculate the amount of parallel lines in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150, apertureSize=3)
    
    # Detect lines using Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    
    if lines is not None:
        return len(lines)  # Return the number of detected lines
    return 0  # No lines detected

def detect_cylinders(image):
    """Detect cylindrical shapes in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (9, 9), 2)

    # Detect circles using Hough Transform
    circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, dp=1, minDist=30,
                               param1=100, param2=30, minRadius=10, maxRadius=100)

    if circles is not None:
        return circles.shape[1]  # Return the number of detected circles
    return 0  # No circles detected

def get_features(image):
    """Get a set of features (yellow amount, silver amount, parallel lines, cylindrical shapes) from the image."""
    yellow_amount = calculate_yellow_amount(image)
    silver_amount = calculate_silver_amount(image)
    parallel_lines = calculate_parallel_lines(image)
    cylinder_count = detect_cylinders(image)  # New feature for cylindrical shapes
    return yellow_amount, silver_amount, parallel_lines, cylinder_count

In [33]:
# Feature extraction testing
test_im = cv2.imread(path + "/garbage-dataset/cardboard/cardboard_25.jpg")
print(get_features(test_im))

(15834, 284114, 140, 15)


## Обучение классификатора

In [ ]:
# Define the path to the dataset
base_path = path + "/garbage-dataset"
classes = ['trash', 'glass', 'battery', 'clothes', 'metal', 'plastic', 'cardboard', 'paper', 'biological', 'shoes']

# Initialize lists to hold features and labels
features = []
labels = []
test_images = []  # List to hold paths of test images

# Loop through each class and extract features from images
for class_name in classes:
    class_path = os.path.join(base_path, class_name)
    image_names = os.listdir(class_path)

    # Split the images into training and testing sets (80% train, 20% test)
    train_images, test_images_class = train_test_split(image_names, test_size=0.2, random_state=42)

    # Process training images
    for image_name in train_images:
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path)

        if image is not None:
            yellow_amount, silver_amount, parallel_lines, cylinder_count = get_features(image)
            features.append([yellow_amount, silver_amount, parallel_lines, cylinder_count])
            labels.append(class_name)

    # Collect test images for manual testing
    for image_name in test_images_class:
        test_images.append(os.path.join(class_path, image_name))

# Convert features and labels to a DataFrame
df = pd.DataFrame(features, columns=['yellow_amount', 'silver_amount', 'parallel_lines', 'cylinder_count'])
df['label'] = labels

# Split the dataset into training and testing sets
X = df[['yellow_amount', 'silver_amount', 'parallel_lines', 'cylinder_count']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Function to classify a new image
def classify_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        yellow_amount, silver_amount, parallel_lines, cylinder_count = get_features(image)
        feature = np.array([[yellow_amount, silver_amount, parallel_lines, cylinder_count]])
        prediction = classifier.predict(feature)
        return prediction[0]
    else:
        return None

In [28]:
# Classifier testing (runs classifier for a specific image)

result = classify_image(path + '/garbage-dataset/metal/metal_2370.jpg') 
print("Predicted class:", result)

Predicted class: glass


In [26]:
# Classifier testing (runs classifier for the remaining images)

def test_classify(test_images, target_class_name):
    predictions = defaultdict(int)
    for test_image in test_images:
        if target_class_name in test_image:  
            predicted_class = classify_image(test_image)  
            predictions[predicted_class] += 1 
    return dict(predictions)  

for class_name in classes:
    results = test_classify(test_images, class_name)
    print("Predicted class counts for test images containing " + class_name + ':')
    for class_name, count in results.items():
        print(f"Class: {class_name}, Count: {count}")

Predicted class counts for test images containing trash:
Class: biological, Count: 6
Class: glass, Count: 36
Class: trash, Count: 51
Class: metal, Count: 4
Class: plastic, Count: 22
Class: battery, Count: 13
Class: shoes, Count: 23
Class: cardboard, Count: 9
Class: clothes, Count: 21
Class: paper, Count: 5
Predicted class counts for test images containing glass:
Class: trash, Count: 37
Class: shoes, Count: 84
Class: glass, Count: 166
Class: clothes, Count: 131
Class: plastic, Count: 49
Class: battery, Count: 25
Class: paper, Count: 24
Class: biological, Count: 35
Class: cardboard, Count: 46
Class: metal, Count: 16
Predicted class counts for test images containing battery:
Class: glass, Count: 31
Class: paper, Count: 13
Class: biological, Count: 5
Class: clothes, Count: 40
Class: battery, Count: 48
Class: plastic, Count: 7
Class: shoes, Count: 23
Class: trash, Count: 5
Class: metal, Count: 6
Class: cardboard, Count: 11
Predicted class counts for test images containing clothes:
Class: cl

## Проверка гипотез